In [42]:
import requests
import pandas as pd
from tqdm import tqdm
import sys,os

import sys
sys.path.insert(0, '../src/misc')
from sql_tools import *
from webscraping_tools import *
from tools import *

In [43]:
url = "https://stats.nba.com/stats/teamgamelogs"

request_headers = 'Accept|*/*&Accept-Encoding|gzip,deflate,br&Accept-Language|en-US,en;q=0.9&Connection|keep-alive&Host|stats.nba.com&Origin|https://www.nba.com&Referer|https://www.nba.com/&Sec-Fetch-Dest|empty&Sec-Fetch-Mode|cors&Sec-Fetch-Site|same-site&User-Agent|Mozilla/5.0(Macintosh;IntelMacOSX10_15_7) AppleWebKit/537.36(KHTML,like Gecko)Chrome/113.0.0.0 Safari/537.36&sec-ch-ua|"GoogleChrome";v="113","Chromium";v="113","Not-A.Brand";v="24"&sec-ch-ua-mobile|?0&sec-ch-ua-platform|"macOS"'
request_headers = clean_api_params(request_headers,"|","&")

In [44]:
seasons = season_range(2017,2022)

for season in tqdm.tqdm(seasons):

    measure_type = ["Base","Advanced"]

    params = query_string_parameters(measure_type[0],season)
    params = clean_api_params(params,"=")

    response = requests.get(url, headers=request_headers, params=params).json()
    base_frame = pd.DataFrame(response['resultSets'][0]['rowSet'],columns = response['resultSets'][0]['headers'])

    params = query_string_parameters(measure_type[1],season)
    params = clean_api_params(params,"=")

    response = requests.get(url, headers=request_headers, params=params).json()
    adv_frame = pd.DataFrame(response['resultSets'][0]['rowSet'],columns = response['resultSets'][0]['headers'])

    frame = base_frame.join(adv_frame.drop(columns=['SEASON_YEAR','TEAM_ID','TEAM_ABBREVIATION','TEAM_NAME','GAME_ID','GAME_DATE','MATCHUP','WL','MIN','GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK']))

    frame[["TEAM_ABBREVIATION","TEAM_NAME","MATCHUP"]] = frame.apply(lambda row: pd.Series(matchup_reformat(row["MATCHUP"])), axis=1) 
    
    frame = frame.rename(columns={"TEAM_ABBREVIATION":"teamAbbr","TEAM_NAME":"opptAbbr","MATCHUP":"location","GAME_DATE":"game_date"})
    
    frame['game_date'] = pd.to_datetime(frame['game_date'])
    #frame = frame.sort_values(by=["GAME_ID","location"]).reset_index(drop=True)
    
    update_database("database.db",f"boxscore_{season}",frame)

 17%|█▋        | 1/6 [00:00<00:03,  1.25it/s]

Table boxscore_2017-18 not found in database database.db. Creating new table...


 33%|███▎      | 2/6 [00:01<00:03,  1.11it/s]

Table boxscore_2018-19 not found in database database.db. Creating new table...


 50%|█████     | 3/6 [00:02<00:02,  1.16it/s]

...

Table boxscore_2020-21 not found in database database.db. Creating new table...


 83%|████████▎ | 5/6 [00:03<00:00,  1.53it/s]

Table boxscore_2021-22 not found in database database.db. Creating new table...


100%|██████████| 6/6 [00:04<00:00,  1.34it/s]

Table boxscore_2022-23 not found in database database.db. Creating new table...


In [45]:
read_database("database.db","boxscore_2022-23")

,SEASON_YEAR,TEAM_ID,teamAbbr,opptAbbr,GAME_ID,game_date,location,WL,MIN,FGM,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,2022-23,1610612744,GSW,POR,0022201230,2023-04-09 00:00:00,away,W,48.0,58,...,46,734,1309,1749,1775,51,1901,2051,627,1410
1,2022-23,1610612751,BKN,PHI,0022201217,2023-04-09 00:00:00,home,L,48.0,35,...,709,2007,706,1150,684,329,1672,1892,627,1051
2,2022-23,1610612765,DET,CHI,0022201223,2023-04-09 00:00:00,away,L,48.0,31,...,837,1261,1908,1205,2052,700,2151,2260,803,2002
3,2022-23,1610612742,DAL,SAS,0022201224,2023-04-09 00:00:00,home,L,48.0,46,...,1795,1223,1850,1590,1849,1848,1053,1047,459,1092
4,2022-23,1610612749,MIL,TOR,0022201221,2023-04-09 00:00:00,away,L,48.0,38,...,1434,316,1635,1103,748,2070,162,200,1049,583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,2022-23,1610612764,WAS,IND,0022200004,2022-10-19 00:00:00,away,W,48.0,42,...,1835,2246,713,335,649,1035,1938,1872,1335,1460
2456,2022-23,1610612755,PHI,BOS,0022200001,2022-10-18 00:00:00,away,L,48.0,40,...,2258,2346,2150,970,1976,1316,731,501,1635,1980
2457,2022-23,1610612738,BOS,PHI,0022200001,2022-10-18 00:00:00,home,W,48.0,46,...,2354,2022,1594,1268,1552,2291,2093,2110,7,1774
2458,2022-23,1610612744,GSW,LAL,0022200002,2022-10-18 00:00:00,home,W,48.0,45,...,1364,891,1183,863,906,1659,1352,1495,7,687


# Odds Data

In [75]:
import requests

def season_format(season):
    return str(season)+'-'+str(season+1)[-2:]

url = "https://www.rotowire.com/betting/nba/tables/games-archive.php"
odds_page = requests.get(url).json()
odds_data = pd.DataFrame(odds_page,columns = odds_page[0].keys())

odds_data['game_date'] = pd.to_datetime(odds_data['game_date'])
odds_data = odds_data[['season','game_date','home_team_abbrev','visit_team_abbrev','game_over_under','spread']]
odds_data['season'] = odds_data['season'].apply(lambda x: season_format(int(x)))

odds_data = odds_data.rename(columns={"home_team_abbrev":"teamAbbr","visit_team_abbrev":"opptAbbr","game_over_under":"O/U_line"})

odds_data

,season,game_date,teamAbbr,opptAbbr,O/U_line,spread
0,2017-18,2017-10-17,CLE,BOS,216.0,4.5
1,2017-18,2017-10-17,GSW,HOU,231.0,9.5
2,2017-18,2017-10-18,DET,CHA,202.0,2.5
3,2017-18,2017-10-18,IND,BKN,216.5,3.0
4,2017-18,2017-10-18,ORL,MIA,205.5,3.5
...,...,...,...,...,...,...
7537,2022-23,2023-06-01,DEN,MIA,219.0,9.0
7538,2022-23,2023-06-04,DEN,MIA,216.0,8.0
7539,2022-23,2023-06-07,MIA,DEN,212.5,3.5
7540,2022-23,2023-06-09,MIA,DEN,211.5,2.5


In [76]:
update_database("database.db","odds",odds_data)

Table odds not found in database database.db. Creating new table...


In [77]:
odds = read_database("database.db","odds")

In [104]:
season_data = read_database('database.db','boxscore_2022-23')
season_data['teamAbbr'] = season_data['teamAbbr'].apply(lambda row: row.replace(" ",""))
season_data['opptAbbr'] = season_data['opptAbbr'].apply(lambda row: row.replace(" ",""))



In [105]:
season_data.loc[(season_data["teamAbbr"] == 'GSW') & (season_data['opptAbbr'] == 'POR')].head(10)

,SEASON_YEAR,TEAM_ID,teamAbbr,opptAbbr,GAME_ID,game_date,location,WL,MIN,FGM,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,2022-23,1610612744,GSW,POR,0022201230,2023-04-09 00:00:00,away,W,48.0,58,...,46,734,1309,1749,1775,51,1901,2051,627,1410
617,2022-23,1610612744,GSW,POR,0022200931,2023-02-28 00:00:00,home,W,48.0,49,...,756,1188,166,331,426,945,2042,2193,2253,1279
817,2022-23,1610612744,GSW,POR,0022200830,2023-02-08 00:00:00,away,L,48.0,46,...,1391,538,496,871,191,1985,595,606,1635,409
1392,2022-23,1610612744,GSW,POR,0022200536,2022-12-30 00:00:00,home,W,48.0,42,...,709,1976,1594,1842,2260,293,1429,1693,245,2206


In [122]:
seasons = season_range(2017,2022)
master_boxscore = pd.DataFrame(read_database('database.db', f'boxscore_{seasons[0]}'))

if len(seasons) != 1: 
    for season in seasons[1:]:
        master_boxscore = pd.concat([master_boxscore,read_database('database.db', f'boxscore_{season}')], axis=0)
        
master_boxscore = master_boxscore.sort_values(by = ['game_date'],ascending=False).reset_index(drop=True)
    
master_odds = read_database('database.db', "odds").sort_values(by = ['game_date'],ascending=False).reset_index(drop=True)

master_odds['teamAbbr'] = master_odds['teamAbbr'].apply(lambda row: row.replace(" ",""))
master_odds['opptAbbr'] = master_odds['opptAbbr'].apply(lambda row: row.replace(" ",""))

master_boxscore['teamAbbr'] = master_boxscore['teamAbbr'].apply(lambda row: row.replace(" ",""))
master_boxscore['opptAbbr'] = master_boxscore['opptAbbr'].apply(lambda row: row.replace(" ",""))

In [123]:
master_odds.merge(master_boxscore, on=['teamAbbr','opptAbbr','game_date'])

,season,game_date,teamAbbr,opptAbbr,O/U_line,spread,SEASON_YEAR,TEAM_ID,GAME_ID,location,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,2022-23,2023-04-09 00:00:00,WAS,HOU,231.0,4.0,2022-23,1610612764,0022201222,home,...,868,613,1353,816,1706,1250,1633,1765,1049,1878
1,2022-23,2023-04-09 00:00:00,BKN,PHI,219.0,1.0,2022-23,1610612751,0022201217,home,...,709,2007,706,1150,684,329,1672,1892,627,1051
2,2022-23,2023-04-09 00:00:00,BOS,ATL,224.0,4.5,2022-23,1610612738,0022201216,home,...,1548,1646,1529,1197,1176,1307,1547,1585,245,873
3,2022-23,2023-04-09 00:00:00,CHI,DET,220.5,8.0,2022-23,1610612741,0022201223,home,...,1250,1585,2215,739,2396,867,2310,2366,21,2457
4,2022-23,2023-04-09 00:00:00,CLE,CHA,216.0,6.5,2022-23,1610612739,0022201218,home,...,393,773,1183,73,78,478,945,1073,13,204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,2017-18,2017-10-18 00:00:00,SAC,HOU,216.5,7.0,2017-18,1610612758,0021700013,home,...,586,2007,447,1303,851,181,2012,1645,91,896
7019,2017-18,2017-10-18 00:00:00,DET,CHA,202.0,2.5,2017-18,1610612765,0021700003,home,...,1369,1908,687,476,350,825,696,832,483,794
7020,2017-18,2017-10-18 00:00:00,IND,BKN,216.5,3.0,2017-18,1610612754,0021700004,home,...,1141,562,1975,1758,2106,1591,935,810,483,1665
7021,2017-18,2017-10-17 00:00:00,GSW,HOU,231.0,9.5,2017-18,1610612744,0021700002,home,...,553,677,381,142,383,861,648,879,581,1358


In [119]:
master_boxscore

,SEASON_YEAR,TEAM_ID,teamAbbr,opptAbbr,GAME_ID,game_date,location,WL,MIN,FGM,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,2022-23,1610612740,NOP,MIN,0022201225,2023-04-09 00:00:00,away,L,48.0,40,...,5,1028,2388,1268,2383,3,1831,2188,13,2257
1,2022-23,1610612749,MIL,TOR,0022201221,2023-04-09 00:00:00,away,L,48.0,38,...,1434,316,1635,1103,748,2070,162,200,1049,583
2,2022-23,1610612741,CHI,DET,0022201223,2023-04-09 00:00:00,home,W,48.0,40,...,1250,1585,2215,739,2396,867,2310,2366,21,2457
3,2022-23,1610612759,SAS,DAL,0022201224,2023-04-09 00:00:00,away,W,48.0,51,...,2247,1847,1892,2054,2218,2126,1951,1699,349,2348
4,2022-23,1610612761,TOR,MIL,0022201221,2023-04-09 00:00:00,home,W,48.0,48,...,2195,1600,1167,803,519,2188,1956,1505,917,436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14113,2017-18,1610612738,BOS,MIL,0021700007,2017-10-18 00:00:00,home,L,48.0,39,...,2317,2193,2322,418,1021,1895,2067,1692,957,1896
14114,2017-18,1610612745,HOU,GSW,0021700002,2017-10-17 00:00:00,away,W,48.0,47,...,689,46,2315,2017,2074,1830,101,68,581,1103
14115,2017-18,1610612744,GSW,HOU,0021700002,2017-10-17 00:00:00,home,L,48.0,43,...,553,677,381,142,383,861,648,879,581,1358
14116,2017-18,1610612739,CLE,BOS,0021700001,2017-10-17 00:00:00,home,W,48.0,38,...,689,1001,1649,762,1625,682,2101,2037,1063,1154


In [120]:
master_odds

,season,game_date,teamAbbr,opptAbbr,O/U_line,spread
0,2022-23,2023-06-12 00:00:00,DEN,MIA,210.0,8.0
1,2022-23,2023-06-09 00:00:00,MIA,DEN,211.5,2.5
2,2022-23,2023-06-07 00:00:00,MIA,DEN,212.5,3.5
3,2022-23,2023-06-04 00:00:00,DEN,MIA,216.0,8.0
4,2022-23,2023-06-01 00:00:00,DEN,MIA,219.0,9.0
...,...,...,...,...,...,...
7537,2017-18,2017-10-18 00:00:00,SAC,HOU,216.5,7.0
7538,2017-18,2017-10-18 00:00:00,DET,CHA,202.0,2.5
7539,2017-18,2017-10-18 00:00:00,IND,BKN,216.5,3.0
7540,2017-18,2017-10-17 00:00:00,GSW,HOU,231.0,9.5
